In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
from tqdm import tqdm_notebook
import warnings
from sklearn.preprocessing import LabelEncoder
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [2]:
path = '../dataofweek/'

In [3]:
#提取base_feature
def get_base_feature(df):
    df['time'] = df.START_DATE.apply(lambda x:x[:19])
    df['time'] = pd.to_datetime(df['time'],format='%Y-%m-%d-%H.%M.%S')
    timedelta = df['time'] - pd.datetime(df['time'].dt.year[0],df['time'].dt.month[0],df['time'].dt.day[0],0,0,0)

    df['hour'] = df.START_DATE.apply(lambda x:int(x[11:13]))
    
    df['ten_min'] = timedelta.dt.seconds/600
    df['ten_min'] = df['ten_min'].astype(int)
    
    df['five_min'] = timedelta.dt.seconds/300
    df['five_min'] = df['five_min'].astype(int)
    
    
    df.TRADE_TYPE[df.TRADE_TYPE==21] = 1
    df.TRADE_TYPE[df.TRADE_TYPE==22] = 0
    
    result = df.groupby(['date','TRADE_ADDRESS','TERMINAL_ID','SAM_ID','RECORD_ROW','hour','ten_min','five_min']).TRADE_TYPE.agg(['count','sum']).reset_index()
    
    result['inNums'] = result['sum']
    result['outNums'] = result['count'] - result['sum']
    del result['count'],result['sum']
    
    result.date = pd.to_datetime(result.date)
    result['week'] = result.date.dt.dayofweek+1
    del df
    return result

In [4]:
#根据EDA的分析进行数据的读取
data_list = ['12','13','14','15','16']
data1 = pd.DataFrame()
for i in data_list:
    file = str(i)+'_week.csv'
    df = pd.read_csv(path+file)
    data1 = pd.concat([data1,df],axis=0,ignore_index=True)         

In [5]:
le = LabelEncoder()
data1['TERMINAL_ID'] = le.fit_transform(data1['TERMINAL_ID'].values)
data1['SAM_ID'] = le.fit_transform(data1['SAM_ID'].values)

In [6]:
szy = data1.copy()

In [7]:
data = get_base_feature(szy)

In [8]:
data = data[(data.week!=5)&(data.week!=6)&(data.week!=7)]

In [9]:
data.shape

(9042608, 11)

In [10]:
def time_before_trans(x,dic_):
    if x in dic_.keys():
        return dic_[x]
    else:
        return np.nan
def generate_fea_y(df, day, n):
    df_feature_y   =  df.loc[df.day_idex == day].copy() 
    
    df_feature_y['tmp_10_minutes'] = df_feature_y['TRADE_ADDRESS'].values * 1000 + df_feature_y['ten_min'].values
    df_feature_y['tmp_hours']      = df_feature_y['TRADE_ADDRESS'].values * 1000 + df_feature_y['hour'].values
    
    for i in range(n): # 前n天每一天
        d = day - i - 1
        df_d = df.loc[df.day_idex == d].copy() # 当天的数据
        
        # 特征1：过去在该时间段（一样的时间段,10minutes）有多少出入量
        df_d['tmp_10_minutes'] = df['TRADE_ADDRESS'] * 1000 + df['ten_min']  
        df_d['tmp_hours']      = df['TRADE_ADDRESS'] * 1000 + df['hour']
        
        # sum
        dic_innums  = df_d.groupby(['tmp_10_minutes'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_10_minutes'])['outNums'].sum().to_dict()
        
        df_feature_y['_bf_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].map(dic_innums).values
        df_feature_y['_bf_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].map(dic_outnums).values
        
        # 特征2：过去在该时间段（小时）有多少出入量
        dic_innums  = df_d.groupby(['tmp_hours'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours'])['outNums'].sum().to_dict()   
        df_feature_y['_bf_' + str(day -d) + '_innum_hour']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf_' + str(day -d) + '_outnum_hour'] =  df_feature_y['tmp_hours'].map(dic_outnums).values   
        
        # mean,max,min
        dic_innums  = df_d.groupby(['tmp_hours'])['inNums'].mean().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours'])['outNums'].mean().to_dict() 
        df_feature_y['_bf_' + str(day -d) + '_innum_hour_mean']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf_' + str(day -d) + '_outnum_hour_mean'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        
        dic_innums  = df_d.groupby(['tmp_hours'])['inNums'].max().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours'])['outNums'].max().to_dict() 
        df_feature_y['_bf_' + str(day -d) + '_innum_hour_max']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf_' + str(day -d) + '_outnum_hour_max'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        
        dic_innums  = df_d.groupby(['tmp_hours'])['inNums'].min().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours'])['outNums'].min().to_dict() 
        df_feature_y['_bf_' + str(day -d) + '_innum_hour_min']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf_' + str(day -d) + '_outnum_hour_min'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        
        # 特征3: 上10分钟
        df_d['tmp_10_minutes_bf'] = df['TRADE_ADDRESS'] * 1000 + df['ten_min'] - 1
        df_d['tmp_hours_bf']      = df['TRADE_ADDRESS'] * 1000 + df['hour'] - 1
        # sum
        dic_innums  = df_d.groupby(['tmp_10_minutes_bf'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_10_minutes_bf'])['outNums'].sum().to_dict()
        df_feature_y['_bf1_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_bf1_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
        
        # 特征4： 上个小时情况
        dic_innums  = df_d.groupby(['tmp_hours_bf'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf'])['outNums'].sum().to_dict()   
        df_feature_y['_bf1_' + str(day -d) + '_innum_hour']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf1_' + str(day -d) + '_outnum_hour'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        # mean,max,min
        dic_innums  = df_d.groupby(['tmp_hours_bf'])['inNums'].mean().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf'])['outNums'].mean().to_dict()   
        df_feature_y['_bf1_' + str(day -d) + '_innum_hour_mean']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf1_' + str(day -d) + '_outnum_hour_mean'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        dic_innums  = df_d.groupby(['tmp_hours_bf'])['inNums'].max().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf'])['outNums'].max().to_dict()   
        df_feature_y['_bf1_' + str(day -d) + '_innum_hour_max']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf1_' + str(day -d) + '_outnum_hour_max'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        dic_innums  = df_d.groupby(['tmp_hours_bf'])['inNums'].min().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf'])['outNums'].min().to_dict()   
        df_feature_y['_bf1_' + str(day -d) + '_innum_hour_min']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf1_' + str(day -d) + '_outnum_hour_min'] =  df_feature_y['tmp_hours'].map(dic_outnums).values
        
        ###############################上20分钟/上2个小时的情况(有提升)##################################################### 
        df_d['tmp_10_minutes_bf2'] = df_d['TRADE_ADDRESS'] * 1000 + df_d['ten_min'] - 2
        df_d['tmp_hours_bf2']      = df_d['TRADE_ADDRESS'] * 1000 + df_d['hour'] - 2
        
        dic_innums  = df_d.groupby(['tmp_10_minutes_bf2'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf2'])['outNums'].sum().to_dict()

        df_feature_y['_bf2_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_bf2_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
        
        dic_innums  = df_d.groupby(['tmp_hours_bf2'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf2'])['outNums'].sum().to_dict()
         
        df_feature_y['_bf2_' + str(day -d) + '_innum_hour']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf2_' + str(day -d) + '_outnum_hour'] =  df_feature_y['tmp_hours'].map(dic_outnums).values 
    
         ###############################后10分钟/下一个小时的情况(有提升)#####################################################
        
        df_d['tmp_10_minutes_la'] = df_d['TRADE_ADDRESS'] * 1000 + df_d['ten_min'] + 1
        df_d['tmp_hours_la']      = df_d['TRADE_ADDRESS'] * 1000 + df_d['hour'] + 1
        
        dic_innums  = df_d.groupby(['tmp_10_minutes_la'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_10_minutes_la'])['outNums'].sum().to_dict()

        df_feature_y['_la1_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_la1_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
        
        dic_innums  = df_d.groupby(['tmp_hours_la'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_la'])['outNums'].sum().to_dict()
         
        df_feature_y['_la1_' + str(day -d) + '_innum_hour']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_la1_' + str(day -d) + '_outnum_hour'] =  df_feature_y['tmp_hours'].map(dic_outnums).values 
        ###############################后10分钟的情况(有提升)#####################################################
        
        df_d['tmp_10_minutes_la2'] = df_d['TRADE_ADDRESS'] * 1000 + df_d['ten_min'] + 2 
        
        dic_innums  = df_d.groupby(['tmp_10_minutes_la2'])['inNums'].sum().to_dict() 
        dic_outnums = df_d.groupby(['tmp_10_minutes_la2'])['outNums'].sum().to_dict()

        df_feature_y['_la2_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_la2_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
         
    
    
        ###############################后30分钟/下一个小时的情况(有提升)#####################################################
        
        df_d['tmp_10_minutes_la3'] = df_d['TRADE_ADDRESS'] * 1000 + df_d['ten_min'] + 3  
        
        dic_innums  = df_d.groupby(['tmp_10_minutes_la3'])['inNums'].sum().to_dict() 
        dic_outnums = df_d.groupby(['tmp_10_minutes_la3'])['outNums'].sum().to_dict()

        df_feature_y['_la3_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_la3_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
        
    
        df_d['tmp_10_minutes_bf3'] = df_d['TRADE_ADDRESS'] * 1000 + df_d['ten_min'] - 3
        df_d['tmp_hours_bf3']      = df_d['TRADE_ADDRESS'] * 1000 + df_d['hour'] - 3
        
        dic_innums  = df_d.groupby(['tmp_10_minutes_bf3'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf3'])['outNums'].sum().to_dict()

        df_feature_y['_bf3_' + str(day -d) + '_innum_10minutes']  =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_innums)).values
        df_feature_y['_bf3_' + str(day -d) + '_outnum_10minutes'] =  df_feature_y['tmp_10_minutes'].agg(lambda x: time_before_trans(x,dic_outnums)).values
        
        dic_innums  = df_d.groupby(['tmp_hours_bf3'])['inNums'].sum().to_dict()
        dic_outnums = df_d.groupby(['tmp_hours_bf3'])['outNums'].sum().to_dict()
         
        df_feature_y['_bf3_' + str(day -d) + '_innum_hour']  =  df_feature_y['tmp_hours'].map(dic_innums).values
        df_feature_y['_bf3_' + str(day -d) + '_outnum_hour'] =  df_feature_y['tmp_hours'].map(dic_outnums).values 
        
    
    for col in ['tmp_10_minutes','tmp_hours']:
        del df_feature_y[col]
        
    return df_feature_y 

In [11]:
date_list = list(data.date.unique())
day_idex = list(range(21))
day_idex.remove(0)
day_idex


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [12]:
data['day_idex'] = 0
for i in range(20):
    data.loc[data.date==date_list[i], 'day_idex']= data.loc[data.date==date_list[i], 'day_idex'].apply(lambda x:day_idex[i]) 
    
data.week = data.date.dt.dayofweek+1

In [13]:
df_feature_y = pd.DataFrame()
for day in range(5,21):
    df_feature_tmp = generate_fea_y(data, day = day, n=4)
    df_feature_y   = pd.concat([df_feature_y,df_feature_tmp], axis=0, ignore_index=True)

In [14]:
df_feature_y.shape

(7263330, 156)

In [15]:
columns = ['_innum_10minutes','_outnum_10minutes','_innum_hour','_outnum_hour']
# # 过去n天的sum,mean
for i in range(2,4):
    for f in columns:
        colname1 = '_bf_'+str(i)+'_'+'days'+f+'_sum'
        df_feature_y[colname1] = 0
        
        for d in range(1,i+1):
            df_feature_y[colname1] = df_feature_y[colname1] + df_feature_y['_bf_'+str(d) +f]
            
        colname2 = '_bf_'+str(d)+'_'+'days'+f+'_mean'
        df_feature_y[colname2] = df_feature_y[colname1] / i
        
        
# 过去n天的mean的差分
for i in range(2,4): 
    for f in columns:
        colname1 = '_bf_'+str(d)+'_'+'days'+f+'_mean'
        colname2 = '_bf_'+str(d)+'_'+'days'+f+'_mean_diff'
        df_feature_y[colname2] = df_feature_y[colname1].diff(1)
        df_feature_y.loc[(df_feature_y.hour==0)&(df_feature_y.ten_min==0), colname2] = 0

In [16]:
columns = ['_innum_10minutes','_outnum_10minutes','_innum_hour','_outnum_hour']
# # 过去n天的sum,mean
for i in range(2,5):
    for f in columns:
        colname1 = '_bf_'+str(i)+'_'+'days'+f+'_sum'
        df_feature_y[colname1] = 0
        
        for d in range(1,i+1):
            df_feature_y[colname1] = df_feature_y[colname1] + df_feature_y['_bf_'+str(d) +f]
            
        colname2 = '_bf_'+str(d)+'_'+'days'+f+'_mean'
        df_feature_y[colname2] = df_feature_y[colname1] / i
        
        
# 过去n天的mean的差分
for i in range(2,5): 
    for f in columns:
        colname1 = '_bf_'+str(d)+'_'+'days'+f+'_mean'
        colname2 = '_bf_'+str(d)+'_'+'days'+f+'_mean_diff'
        df_feature_y[colname2] = df_feature_y[colname1].diff(1)
        df_feature_y.loc[(df_feature_y.hour==0)&(df_feature_y.ten_min==0), colname2] = 0

In [17]:
def get_stationID_fea(df):
    df_station               = pd.DataFrame()
    df_station['TRADE_ADDRESS']  = df['TRADE_ADDRESS'].unique()
    df_station               = df_station.sort_values('TRADE_ADDRESS')
    
    tmp1  = df.groupby(['TRADE_ADDRESS'])['TERMINAL_ID'].nunique().to_frame('TRADE_ADDRESS_TERMINAL_ID_nunique').reset_index()
    tmp2  = df.groupby(['TRADE_ADDRESS'])['CARD_ID'].nunique().to_frame('TRADE_ADDRESS_CARD_ID_nunique').reset_index()
    tmp3  = df.groupby(['TRADE_ADDRESS'])['FILE_ID'].nunique().to_frame('TRADE_ADDRESS_FILE_ID_nunique').reset_index()
    tmp4  = df.groupby(['TRADE_ADDRESS'])['CARD_SN'].nunique().to_frame('TRADE_ADDRESS_CARD_SN_nunique').reset_index()
    tmp5  = df.groupby(['TRADE_ADDRESS'])['TERMINAL_SN'].nunique().to_frame('TRADE_ADDRESS_TERMINAL_SN_nunique').reset_index()
    tmp6  = df.groupby(['TRADE_ADDRESS'])['TAC'].nunique().to_frame('TRADE_ADDRESS_TAC_nunique').reset_index()
    tmp7  = df.groupby(['TRADE_ADDRESS'])['SAM_ID'].nunique().to_frame('TRADE_ADDRESS_SAM_ID_nunique').reset_index()
    tmp8  = df.groupby(['TRADE_ADDRESS'])['SAM_SN'].nunique().to_frame('TRADE_ADDRESS_SAM_SN_nunique').reset_index()
    tmp9  = df.groupby(['TRADE_ADDRESS'])['CARD_IDM'].nunique().to_frame('TRADE_ADDRESS_CARD_IDM_nunique').reset_index()

    df_station = df_station.merge(tmp1,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp2,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp3,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp4,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp5,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp6,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp7,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp8,on ='TRADE_ADDRESS', how='left')
    df_station = df_station.merge(tmp9,on ='TRADE_ADDRESS', how='left')

    
    for pivot_cols in tqdm_notebook(['CARD_TYPE','CARD_TYPE_EX','CITY_CODE','INDUSTRY_CODE','hour','week']):
        tmp = df.groupby(['TRADE_ADDRESS',pivot_cols])['TERMINAL_ID'].count().to_frame('TRADE_ADDRESS'+pivot_cols+'_cnt').reset_index()
        df_tmp = tmp.pivot(index = 'TRADE_ADDRESS', columns=pivot_cols, values='TRADE_ADDRESS'+pivot_cols+'_cnt')
        cols   = ['TRADE_ADDRESS'+pivot_cols+'_cnt' + str(col) for col in df_tmp.columns]
        df_tmp.columns = cols
        df_tmp.reset_index(inplace = True)
        df_station = df_station.merge(df_tmp, on ='TRADE_ADDRESS', how='left')
    return df_station

In [18]:
len(list(szy.CARD_TYPE_EX.unique()))

9

In [19]:
data_list = ['14']
df_station = pd.DataFrame()
for i in data_list:
    file = str(i)+'_week.csv'
    df = pd.read_csv(path+file)
    df_station = pd.concat([df_station,df],axis=0,ignore_index=True)
    
df_station['time'] = df_station.START_DATE.apply(lambda x:x[:19])
df_station['time'] = pd.to_datetime(df_station['time'],format='%Y-%m-%d-%H.%M.%S')
timedelta = df_station['time'] - pd.datetime(df_station['time'].dt.year[0],df_station['time'].dt.month[0],df_station['time'].dt.day[0],0,0,0)
df_station['hour'] = df_station.START_DATE.apply(lambda x:int(x[11:13]))
df_station['ten_min'] = timedelta.dt.seconds/600
df_station['ten_min'] = df_station['ten_min'].astype(int)
df_station.date = pd.to_datetime(df_station.date)
df_station['week'] = df_station.date.dt.dayofweek+1

In [20]:
df_station = get_stationID_fea(df_station)
df_station.fillna(0,inplace=True)

In [21]:
data = df_feature_y.copy()

In [22]:
data = data.merge(df_station,on=['TRADE_ADDRESS'],how='left')
data.fillna(0,inplace=True)

In [23]:
all_columns = ['TRADE_ADDRESS',
 'TERMINAL_ID',
 'SAM_ID',
 'RECORD_ROW',
 'hour',
 'ten_min',
 'five_min',
 'inNums',
 'outNums',
 'week',
 '_bf_1_innum_10minutes',
 '_bf_1_outnum_10minutes',
 '_bf_1_innum_hour',
 '_bf_1_outnum_hour',
 '_bf_1_innum_hour_mean',
 '_bf_1_outnum_hour_mean',
 '_bf_1_outnum_hour_max',
 '_bf1_1_innum_10minutes',
 '_bf1_1_outnum_10minutes',
 '_bf1_1_innum_hour',
 '_bf1_1_outnum_hour',
 '_bf1_1_innum_hour_mean',
 '_bf1_1_outnum_hour_mean',
 '_bf1_1_outnum_hour_max',
 '_bf2_1_innum_10minutes',
 '_bf2_1_innum_hour',
 '_bf2_1_outnum_hour',
 '_la1_1_innum_10minutes',
 '_la1_1_outnum_10minutes',
 '_la1_1_innum_hour',
 '_la1_1_outnum_hour',
 '_la2_1_innum_10minutes',
 '_la2_1_outnum_10minutes',
 '_la3_1_innum_10minutes',
 '_la3_1_outnum_10minutes',
 '_bf3_1_innum_10minutes',
 '_bf3_1_innum_hour',
 '_bf3_1_outnum_hour',
 '_bf_2_innum_10minutes',
 '_bf_2_outnum_10minutes',
 '_bf_2_innum_hour',
 '_bf_2_outnum_hour',
 '_bf_2_innum_hour_mean',
 '_bf_2_outnum_hour_mean',
 '_bf_2_outnum_hour_max',
 '_bf1_2_innum_10minutes',
 '_bf1_2_outnum_10minutes',
 '_bf1_2_innum_hour',
 '_bf1_2_outnum_hour',
 '_bf1_2_innum_hour_mean',
 '_bf1_2_outnum_hour_mean',
 '_bf1_2_outnum_hour_max',
 '_bf2_2_innum_10minutes',
 '_bf2_2_innum_hour',
 '_bf2_2_outnum_hour',
 '_la1_2_innum_10minutes',
 '_la1_2_outnum_10minutes',
 '_la1_2_innum_hour',
 '_la1_2_outnum_hour',
 '_la2_2_innum_10minutes',
 '_la2_2_outnum_10minutes',
 '_la3_2_innum_10minutes',
 '_la3_2_outnum_10minutes',
 '_bf3_2_innum_10minutes',
 '_bf3_2_innum_hour',
 '_bf3_2_outnum_hour',
 '_bf_3_innum_10minutes',
 '_bf_3_outnum_10minutes',
 '_bf_3_innum_hour',
 '_bf_3_outnum_hour',
 '_bf_3_innum_hour_mean',
 '_bf_3_outnum_hour_mean',
 '_bf_3_outnum_hour_max',
 '_bf1_3_innum_10minutes',
 '_bf1_3_outnum_10minutes',
 '_bf1_3_innum_hour',
 '_bf1_3_outnum_hour',
 '_bf1_3_innum_hour_mean',
 '_bf1_3_outnum_hour_mean',
 '_bf1_3_outnum_hour_max',
 '_bf2_3_innum_10minutes',
 '_bf2_3_innum_hour',
 '_bf2_3_outnum_hour',
 '_la1_3_innum_10minutes',
 '_la1_3_outnum_10minutes',
 '_la1_3_innum_hour',
 '_la1_3_outnum_hour',
 '_la2_3_innum_10minutes',
 '_la2_3_outnum_10minutes',
 '_la3_3_innum_10minutes',
 '_la3_3_outnum_10minutes',
 '_bf3_3_innum_10minutes',
 '_bf3_3_innum_hour',
 '_bf3_3_outnum_hour',
 '_bf_4_innum_10minutes',
 '_bf_4_outnum_10minutes',
 '_bf_4_innum_hour',
 '_bf_4_outnum_hour',
 '_bf_4_innum_hour_mean',
 '_bf_4_outnum_hour_mean',
 '_bf_4_outnum_hour_max',
 '_bf1_4_innum_10minutes',
 '_bf1_4_outnum_10minutes',
 '_bf1_4_innum_hour',
 '_bf1_4_outnum_hour',
 '_bf1_4_innum_hour_mean',
 '_bf1_4_outnum_hour_mean',
 '_bf1_4_outnum_hour_max',
 '_bf2_4_innum_10minutes',
 '_bf2_4_innum_hour',
 '_bf2_4_outnum_hour',
 '_la1_4_innum_10minutes',
 '_la1_4_outnum_10minutes',
 '_la1_4_innum_hour',
 '_la1_4_outnum_hour',
 '_la2_4_innum_10minutes',
 '_la2_4_outnum_10minutes',
 '_la3_4_innum_10minutes',
 '_la3_4_outnum_10minutes',
 '_bf3_4_innum_10minutes',
 '_bf3_4_innum_hour',
 '_bf3_4_outnum_hour',
 '_bf_2_days_innum_10minutes_sum',
 '_bf_2_days_innum_10minutes_mean',
 '_bf_2_days_outnum_10minutes_sum',
 '_bf_2_days_outnum_10minutes_mean',
 '_bf_2_days_innum_hour_sum',
 '_bf_2_days_innum_hour_mean',
 '_bf_2_days_outnum_hour_sum',
 '_bf_2_days_outnum_hour_mean',
 '_bf_3_days_innum_10minutes_sum',
 '_bf_3_days_innum_10minutes_mean',
 '_bf_3_days_outnum_10minutes_sum',
 '_bf_3_days_outnum_10minutes_mean',
 '_bf_3_days_innum_hour_sum',
 '_bf_3_days_innum_hour_mean',
 '_bf_3_days_outnum_hour_sum',
 '_bf_3_days_outnum_hour_mean',
 '_bf_3_days_innum_10minutes_mean_diff',
 '_bf_3_days_outnum_10minutes_mean_diff',
 '_bf_3_days_innum_hour_mean_diff',
 '_bf_3_days_outnum_hour_mean_diff',
 '_bf_4_days_innum_10minutes_sum',
 '_bf_4_days_innum_10minutes_mean',
 '_bf_4_days_outnum_10minutes_sum',
 '_bf_4_days_outnum_10minutes_mean',
 '_bf_4_days_innum_hour_sum',
 '_bf_4_days_innum_hour_mean',
 '_bf_4_days_outnum_hour_sum',
 '_bf_4_days_outnum_hour_mean',
 '_bf_4_days_innum_10minutes_mean_diff',
 '_bf_4_days_outnum_10minutes_mean_diff',
 '_bf_4_days_innum_hour_mean_diff',
 '_bf_4_days_outnum_hour_mean_diff',
 'TRADE_ADDRESS_TERMINAL_ID_nunique',
 'TRADE_ADDRESS_CARD_ID_nunique',
 'TRADE_ADDRESS_FILE_ID_nunique',
 'TRADE_ADDRESS_CARD_SN_nunique',
 'TRADE_ADDRESS_TERMINAL_SN_nunique',
 'TRADE_ADDRESS_TAC_nunique',
 'TRADE_ADDRESS_SAM_ID_nunique',
 'TRADE_ADDRESS_SAM_SN_nunique',
 'TRADE_ADDRESSCARD_TYPE_cnt50',
 'TRADE_ADDRESSCARD_TYPE_cnt66',
 'TRADE_ADDRESSCARD_TYPE_cnt81',
 'TRADE_ADDRESSCARD_TYPE_cnt87',
 'TRADE_ADDRESSCARD_TYPE_cnt88',
 'TRADE_ADDRESSCARD_TYPE_cnt95',
 'TRADE_ADDRESSCARD_TYPE_cnt98',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt1',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt2',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt3',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt4',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt5',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt6',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt7',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt90',
 'TRADE_ADDRESSCITY_CODE_cnt1100',
 'TRADE_ADDRESSCITY_CODE_cnt2260',
 'TRADE_ADDRESSCITY_CODE_cnt3000',
 'TRADE_ADDRESSCITY_CODE_cnt3300',
 'TRADE_ADDRESSCITY_CODE_cnt4500',
 'TRADE_ADDRESSINDUSTRY_CODE_cnt1',
 'TRADE_ADDRESSINDUSTRY_CODE_cnt3',
 'TRADE_ADDRESShour_cnt5',
 'TRADE_ADDRESShour_cnt6',
 'TRADE_ADDRESShour_cnt7',
 'TRADE_ADDRESShour_cnt8',
 'TRADE_ADDRESShour_cnt10',
 'TRADE_ADDRESShour_cnt13',
 'TRADE_ADDRESShour_cnt17',
 'TRADE_ADDRESShour_cnt18',
 'TRADE_ADDRESShour_cnt19',
 'TRADE_ADDRESShour_cnt20',
 'TRADE_ADDRESShour_cnt21',
 'TRADE_ADDRESShour_cnt22',
 'TRADE_ADDRESShour_cnt23',
 'TRADE_ADDRESSweek_cnt1',
 'TRADE_ADDRESSweek_cnt2']

In [24]:
#lgb参数
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2,
    'min_child_weight':6
}

In [25]:
#########inNums
# train_in = data[data.id_week!=4]
train_in = data[data.day_idex<13]
train_in = train_in[all_columns]
y_train_in = train_in['inNums']

del train_in['outNums'],train_in['inNums']

valid = data[(data.day_idex>=13)&(data.day_idex<=16)]
valid = valid[all_columns]
y_valid = valid['inNums']
del valid['outNums'],valid['inNums']
# y_valid = np.log1p(y_valid)

test_in = data[data.day_idex>=17]
test_in = test_in[all_columns]
y_test_in = test_in['inNums']
del test_in['inNums'],test_in['outNums']

In [26]:
train_in.shape

(3608713, 197)

In [27]:
valid.shape

(1818654, 197)

In [28]:
test = data[data.day_idex>=17]

In [ ]:
dtrain = xgb.DMatrix(train_in, label = y_train_in)
dtest = xgb.DMatrix(test_in)
dval = xgb.DMatrix(valid, label = y_valid)
watchlist = [(dtrain, 'train'),(dval, 'val')]
xgb_params = {'eta': 0.004, 'max_depth': 14, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'mae', 'silent': True, 'nthread': 4}
clf1 = xgb.train(dtrain=dtrain, num_boost_round=15000, evals=watchlist, early_stopping_rounds=100, verbose_eval=100, params=xgb_params)      

[0]	train-mae:0.498558	val-mae:0.498512
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 100 rounds.
[100]	train-mae:0.385465	val-mae:0.383204
[200]	train-mae:0.299676	val-mae:0.296148
[300]	train-mae:0.235681	val-mae:0.231864
[400]	train-mae:0.186029	val-mae:0.182236
[500]	train-mae:0.149766	val-mae:0.146973
[600]	train-mae:0.122376	val-mae:0.120562
[700]	train-mae:0.09934	val-mae:0.098541
[800]	train-mae:0.083976	val-mae:0.083879
[900]	train-mae:0.070754	val-mae:0.071316
[1000]	train-mae:0.05857	val-mae:0.059718
[1100]	train-mae:0.048829	val-mae:0.050606
[1200]	train-mae:0.040908	val-mae:0.04337
[1300]	train-mae:0.035208	val-mae:0.038264
[1400]	train-mae:0.030568	val-mae:0.034188
[1500]	train-mae:0.027012	val-mae:0.031107
[1600]	train-mae:0.024153	val-mae:0.028576
[1700]	train-mae:0.021972	val-mae:0.026668


In [ ]:
prediction_in = clf1.predict(dtest, ntree_limit=clf.best_iteration)

In [ ]:
error_in = mean_absolute_error(prediction_in,y_test_in)
error_in

In [ ]:
test['pre_inNums'] = prediction_in

In [ ]:
all_columns =['TRADE_ADDRESS',
 'TERMINAL_ID',
 'SAM_ID',
 'RECORD_ROW',
 'hour',
 'ten_min',
 'five_min',
 'inNums',
 'outNums',
 'week',
 '_bf_1_innum_10minutes',
 '_bf_1_outnum_10minutes',
 '_bf_1_innum_hour',
 '_bf_1_outnum_hour',
 '_bf_1_innum_hour_mean',
 '_bf_1_outnum_hour_mean',
 '_bf_1_outnum_hour_max',
 '_bf1_1_innum_10minutes',
 '_bf1_1_outnum_10minutes',
 '_bf1_1_innum_hour',
 '_bf1_1_outnum_hour',
 '_bf1_1_innum_hour_mean',
 '_bf1_1_outnum_hour_mean',
 '_bf1_1_outnum_hour_max',
 '_bf2_1_innum_10minutes',
 '_bf2_1_innum_hour',
 '_bf2_1_outnum_hour',
 '_la1_1_innum_10minutes',
 '_la1_1_outnum_10minutes',
 '_la1_1_innum_hour',
 '_la1_1_outnum_hour',
 '_la2_1_innum_10minutes',
 '_la2_1_outnum_10minutes',
 '_la3_1_innum_10minutes',
 '_la3_1_outnum_10minutes',
 '_bf3_1_innum_10minutes',
 '_bf3_1_innum_hour',
 '_bf3_1_outnum_hour',
 '_bf_2_innum_10minutes',
 '_bf_2_outnum_10minutes',
 '_bf_2_innum_hour',
 '_bf_2_outnum_hour',
 '_bf_2_innum_hour_mean',
 '_bf_2_outnum_hour_mean',
 '_bf_2_outnum_hour_max',
 '_bf1_2_innum_10minutes',
 '_bf1_2_outnum_10minutes',
 '_bf1_2_innum_hour',
 '_bf1_2_outnum_hour',
 '_bf1_2_innum_hour_mean',
 '_bf1_2_outnum_hour_mean',
 '_bf1_2_outnum_hour_max',
 '_bf2_2_innum_10minutes',
 '_bf2_2_innum_hour',
 '_bf2_2_outnum_hour',
 '_la1_2_innum_10minutes',
 '_la1_2_outnum_10minutes',
 '_la1_2_innum_hour',
 '_la1_2_outnum_hour',
 '_la2_2_innum_10minutes',
 '_la2_2_outnum_10minutes',
 '_la3_2_innum_10minutes',
 '_la3_2_outnum_10minutes',
 '_bf3_2_innum_10minutes',
 '_bf3_2_innum_hour',
 '_bf3_2_outnum_hour',
 '_bf_3_innum_10minutes',
 '_bf_3_outnum_10minutes',
 '_bf_3_innum_hour',
 '_bf_3_outnum_hour',
 '_bf_3_innum_hour_mean',
 '_bf_3_outnum_hour_mean',
 '_bf_3_outnum_hour_max',
 '_bf1_3_innum_10minutes',
 '_bf1_3_outnum_10minutes',
 '_bf1_3_innum_hour',
 '_bf1_3_outnum_hour',
 '_bf1_3_innum_hour_mean',
 '_bf1_3_outnum_hour_mean',
 '_bf1_3_outnum_hour_max',
 '_bf2_3_innum_10minutes',
 '_bf2_3_innum_hour',
 '_bf2_3_outnum_hour',
 '_la1_3_innum_10minutes',
 '_la1_3_outnum_10minutes',
 '_la1_3_innum_hour',
 '_la1_3_outnum_hour',
 '_la2_3_innum_10minutes',
 '_la2_3_outnum_10minutes',
 '_la3_3_innum_10minutes',
 '_la3_3_outnum_10minutes',
 '_bf3_3_innum_10minutes',
 '_bf3_3_innum_hour',
 '_bf3_3_outnum_hour',
 '_bf_4_innum_10minutes',
 '_bf_4_outnum_10minutes',
 '_bf_4_innum_hour',
 '_bf_4_outnum_hour',
 '_bf_4_innum_hour_mean',
 '_bf_4_outnum_hour_mean',
 '_bf_4_outnum_hour_max',
 '_bf1_4_innum_10minutes',
 '_bf1_4_outnum_10minutes',
 '_bf1_4_innum_hour',
 '_bf1_4_outnum_hour',
 '_bf1_4_innum_hour_mean',
 '_bf1_4_outnum_hour_mean',
 '_bf1_4_outnum_hour_max',
 '_bf2_4_innum_10minutes',
 '_bf2_4_innum_hour',
 '_bf2_4_outnum_hour',
 '_la1_4_innum_10minutes',
 '_la1_4_outnum_10minutes',
 '_la1_4_innum_hour',
 '_la1_4_outnum_hour',
 '_la2_4_innum_10minutes',
 '_la2_4_outnum_10minutes',
 '_la3_4_innum_10minutes',
 '_la3_4_outnum_10minutes',
 '_bf3_4_innum_10minutes',
 '_bf3_4_innum_hour',
 '_bf3_4_outnum_hour',
 '_bf_2_days_innum_10minutes_sum',
 '_bf_2_days_innum_10minutes_mean',
 '_bf_2_days_outnum_10minutes_sum',
 '_bf_2_days_outnum_10minutes_mean',
 '_bf_2_days_innum_hour_sum',
 '_bf_2_days_innum_hour_mean',
 '_bf_2_days_outnum_hour_sum',
 '_bf_2_days_outnum_hour_mean',
 '_bf_3_days_innum_10minutes_sum',
 '_bf_3_days_innum_10minutes_mean',
 '_bf_3_days_outnum_10minutes_sum',
 '_bf_3_days_outnum_10minutes_mean',
 '_bf_3_days_innum_hour_sum',
 '_bf_3_days_innum_hour_mean',
 '_bf_3_days_outnum_hour_sum',
 '_bf_3_days_outnum_hour_mean',
 '_bf_3_days_innum_10minutes_mean_diff',
 '_bf_3_days_outnum_10minutes_mean_diff',
 '_bf_3_days_innum_hour_mean_diff',
 '_bf_3_days_outnum_hour_mean_diff',
 '_bf_4_days_innum_10minutes_sum',
 '_bf_4_days_innum_10minutes_mean',
 '_bf_4_days_outnum_10minutes_sum',
 '_bf_4_days_outnum_10minutes_mean',
 '_bf_4_days_innum_hour_sum',
 '_bf_4_days_innum_hour_mean',
 '_bf_4_days_outnum_hour_sum',
 '_bf_4_days_outnum_hour_mean',
 '_bf_4_days_innum_10minutes_mean_diff',
 '_bf_4_days_outnum_10minutes_mean_diff',
 '_bf_4_days_innum_hour_mean_diff',
 '_bf_4_days_outnum_hour_mean_diff',
 'TRADE_ADDRESS_TERMINAL_ID_nunique',
 'TRADE_ADDRESS_CARD_ID_nunique',
 'TRADE_ADDRESS_FILE_ID_nunique',
 'TRADE_ADDRESS_CARD_SN_nunique',
 'TRADE_ADDRESS_TERMINAL_SN_nunique',
 'TRADE_ADDRESS_TAC_nunique',
 'TRADE_ADDRESS_SAM_ID_nunique',
 'TRADE_ADDRESS_SAM_SN_nunique',
 'TRADE_ADDRESSCARD_TYPE_cnt50',
 'TRADE_ADDRESSCARD_TYPE_cnt66',
 'TRADE_ADDRESSCARD_TYPE_cnt81',
 'TRADE_ADDRESSCARD_TYPE_cnt87',
 'TRADE_ADDRESSCARD_TYPE_cnt88',
 'TRADE_ADDRESSCARD_TYPE_cnt95',
 'TRADE_ADDRESSCARD_TYPE_cnt98',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt1',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt2',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt3',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt4',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt5',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt6',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt7',
 'TRADE_ADDRESSCARD_TYPE_EX_cnt90',
 'TRADE_ADDRESSCITY_CODE_cnt1100',
 'TRADE_ADDRESSCITY_CODE_cnt2260',
 'TRADE_ADDRESSCITY_CODE_cnt3000',
 'TRADE_ADDRESSCITY_CODE_cnt3300',
 'TRADE_ADDRESSCITY_CODE_cnt4500',
 'TRADE_ADDRESSINDUSTRY_CODE_cnt1',
 'TRADE_ADDRESSINDUSTRY_CODE_cnt3',
 'TRADE_ADDRESShour_cnt5',
 'TRADE_ADDRESShour_cnt6',
 'TRADE_ADDRESShour_cnt7',
 'TRADE_ADDRESShour_cnt8',
 'TRADE_ADDRESShour_cnt10',
 'TRADE_ADDRESShour_cnt13',
 'TRADE_ADDRESShour_cnt17',
 'TRADE_ADDRESShour_cnt18',
 'TRADE_ADDRESShour_cnt19',
 'TRADE_ADDRESShour_cnt20',
 'TRADE_ADDRESShour_cnt21',
 'TRADE_ADDRESShour_cnt22',
 'TRADE_ADDRESShour_cnt23',
 'TRADE_ADDRESSweek_cnt1',
 'TRADE_ADDRESSweek_cnt2']

In [ ]:
#########inNums
# train_out = data[data.id_week!=4]
train_out = data[data.day_idex<13]
train_out = train_out[all_columns]
y_train_out = train_out['outNums']
del train_out['outNums'],train_out['inNums']

valid = data[(data.day_idex>=13)&(data.day_idex<=16)]
valid = valid[all_columns]
y_valid = valid['outNums']
del valid['outNums'],valid['inNums']
# y_valid = np.log1p(y_valid)

test_out = data[data.day_idex>=17]
test_out = test_out[all_columns]
y_test_out = test_out['outNums']
del test_out['inNums'],test_out['outNums']

In [ ]:
dtrain = xgb.DMatrix(train_out, label = y_train_out)
dtest = xgb.DMatrix(test_out)
dval = xgb.DMatrix(valid, label = y_valid)
watchlist = [(dtrain, 'train'),(dval, 'val')]
xgb_params = {'eta': 0.004, 'max_depth': 14, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'mae', 'silent': True, 'nthread': 4}
clf2 = xgb.train(dtrain=dtrain, num_boost_round=30000, evals=watchlist, early_stopping_rounds=100, verbose_eval=1000, params=xgb_params)

In [ ]:
prediction_out = clf2.predict(dtest, ntree_limit=clf.best_iteration)

In [ ]:
error_out = mean_absolute_error(prediction_out,y_test_out)
error_out

In [ ]:
test['pre_outNums'] = prediction_out

In [ ]:
sub1 = test.groupby(['date','TRADE_ADDRESS']).pre_inNums.agg({'inNums':'sum'}).reset_index()
sub2 = test.groupby(['date','TRADE_ADDRESS']).pre_outNums.agg({'outNums':'sum'}).reset_index()
sub = sub1.merge(sub2,on = ['date','TRADE_ADDRESS'],how='left')
sub['flow'] = sub.inNums + sub.outNums
sub['round_pre'] = np.round(sub.flow)
sub['round_in'] = np.round(sub.inNums)
sub['round_out'] = np.round(sub.outNums)

In [ ]:
real1 = test.groupby(['date','TRADE_ADDRESS']).inNums.agg({'inNums':'sum'}).reset_index()
real2 = test.groupby(['date','TRADE_ADDRESS']).outNums.agg({'outNums':'sum'}).reset_index()
real = real1.merge(real2,on = ['date','TRADE_ADDRESS'],how='left')
real['flow'] = real.inNums + real.outNums

In [ ]:
sub.head()

In [ ]:
real.head()

In [ ]:
error = mean_absolute_error(sub.flow,real.flow)
error

In [ ]:
list(sub.columns)

In [ ]:
sub_columns = ['date',
 'TRADE_ADDRESS',
 'round_pre',
 'round_in',
 'round_out']

In [ ]:
sub = sub[sub_columns]

In [ ]:
sub.head()

In [ ]:
re_path = '../result2/'
sub.to_csv(re_path+'1_2_3_4_flow_pre_st2_xgb_'+str('%.2f'%error)+'.csv',encoding='utf-8',index=False)
real.to_csv(re_path+'1_2_3_4_flow_real_st2.csv',encoding='utf-8',index=False)

In [ ]:
day_list = [15,16,17,18,19,20]

In [ ]:
fig = plt.figure(figsize=(40,20))
for i in day_list:
    hh = data[data.day_idex==i]
    hhh = hh.groupby(['hour']).inNums.agg({'inNums':'sum'}).reset_index()
    plt.plot(hhh['hour'],hhh['inNums'],label = i)
    
legend=plt.legend(title='time', fontsize=30)
legend.get_title().set_fontsize(fontsize=30)

plt.tick_params(labelsize=30)
plt.show()